In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import math
from tqdm import tqdm
import requests
from urllib.parse import urlparse
pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 500)
import datetime
from sklearn.preprocessing import quantile_transform
pd.options.mode.chained_assignment = None  # default='warn'
from time import sleep

In [2]:
addr_to_download_df = pd.read_csv('./prepped_data/addr_to_download.csv')
print(addr_to_download_df.shape)
addr_to_download_df.head()

(33720, 1)


,지번주소
0,서울특별시 강남구 청담동 129-16
1,서울특별시 강서구 공항동 49-1
2,서울특별시 노원구 상계동 341-15
3,서울특별시 동대문구 용두동 236-4
4,서울특별시 영등포구 당산동2가 15-2


In [3]:
# download coordinates from an open api

def fetch_addr_data(addr_list):
    roadname_url = 'https://www.juso.go.kr/addrlink/addrLinkApi.do?'
    roadname_apikey = 'devU01TX0FVVEgyMDIxMDUwNDE1NTMwMDExMTEyNzc='
    #apikey 만료시 https://www.juso.go.kr/addrlink/devAddrLinkRequestWrite.do?returnFn=write&cntcMenu=URL 가서 새로 받을 것.
    
    addr_df = pd.DataFrame({'지번주소':[], '도로명주소': []})
    for i, address in tqdm(enumerate(addr_list)):
        address_no_zero = address.replace('-0','')
        
        roadname_result = requests.get(roadname_url + 'confmKey=' + roadname_apikey + '&currentPage=1' + '&countPerPage=1' + '&keyword=' \
                                       + address_no_zero + '&resultType=json' + '&hstryYn=Y')
        roadname_info = roadname_result.json()['results']['juso']
        
        if roadname_info == []:
            addr_df.loc[i,'지번주소'] = address
            addr_df.loc[i,'도로명주소'] = np.nan
            continue
        
        roadname_addr = roadname_info[0]['roadAddrPart1']
        
        addr_df.loc[i,'지번주소'] = address
        addr_df.loc[i,'도로명주소'] = roadname_addr
        
        sleep(1)    
    
    return addr_df

In [4]:
addr_df = fetch_addr_data(addr_to_download_df['지번주소'].tolist())

33720it [1:27:31,  6.42it/s]


In [5]:
print(addr_df.shape)
addr_df.head()

(33720, 2)


,지번주소,도로명주소
0,서울특별시 강남구 청담동 129-16,서울특별시 강남구 압구정로79길 72
1,서울특별시 강서구 공항동 49-1,서울특별시 강서구 공항대로 18-1
2,서울특별시 노원구 상계동 341-15,서울특별시 노원구 상계로 90
3,서울특별시 동대문구 용두동 236-4,서울특별시 동대문구 왕산로 52
4,서울특별시 영등포구 당산동2가 15-2,서울특별시 영등포구 영등포로 112


In [6]:
addr_df.isna().sum()

지번주소         0
도로명주소    26126
dtype: int64

In [7]:
addr_df.to_csv('./prepped_data/additional_addr_1.csv', index=False)